# **Análise de sentimento**
A análise de sentimentos é uma mineração contextual de um texto que identifica e extrai informações subjetivas no material de origem. Ela ajuda as empresas a entenderem o sentimento social de sua marca, produto ou serviço.

Um sistema de análise de sentimentos para conteúdo textual combina o processamento de linguagem natural (PLN) e técnicas de aprendizado de máquina para atribuir pontuações ponderadas de sentimento às sentenças.


---



Primeiro, precisamos importar as bibliotecas que serão utilizadas nesse script


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_predict


Importando a base de dados

In [2]:
base_de_dados = pd.read_csv('https://raw.githubusercontent.com/audreyemmely/DSProjects/master/analise-sentimento/analise-base.csv')

Olhando o existente na base de dados



In [3]:
base_de_dados.head()

,Frase,Sentimento
0,a pandemia me impediu de fazer muitas coisas l...,tristeza
1,"acordei desanimada, não quero fazer nada o dia...",tristeza
2,"não consigo parar de chorar, foi horrível perd...",tristeza
3,me sinto amada pelos meus familiares e isso me...,felicidade
4,estou feliz porque consegui o estágio que quer...,felicidade


Contando a quantidade total de registros

In [4]:
base_de_dados.count()

Frase         66
Sentimento    66
dtype: int64

Contando quantas frases têm sentimento igual a felicidade

In [5]:
base_de_dados[base_de_dados.Sentimento == 'felicidade'].count()

Frase         19
Sentimento    19
dtype: int64

Contando quantas frases têm sentimento igual a raiva

In [6]:
base_de_dados[base_de_dados.Sentimento == 'raiva'].count()

Frase         23
Sentimento    23
dtype: int64

Contando quantas frases têm sentimento igual a tristeza

In [7]:
base_de_dados[base_de_dados.Sentimento == 'tristeza'].count()

Frase         24
Sentimento    24
dtype: int64

### Construindo o modelo

Separando as frases e os sentimentos

In [8]:
frases = base_de_dados['Frase'].values
frases

array(['a pandemia me impediu de fazer muitas coisas legais infelizmente não pude aproveitar esse ano',
       'acordei desanimada, não quero fazer nada o dia todo',
       'não consigo parar de chorar, foi horrível perder um parente próximo por causa de uma pandemia',
       'me sinto amada pelos meus familiares e isso me faz muito bem',
       'estou feliz porque consegui o estágio que queria, agora é só felicidade!',
       'receber você na minha casa é uma alegria imensa, venha sempre que puder',
       'me sinto muito bem hoje, estou disposta, feliz e bem humorada',
       'te odeio pelo que você é, não pelo que faz',
       'só tenho raiva, ódio e rancor',
       'obrigada por me ajudar, fiquei contente com isso',
       'estou contente fui aprovada na faculdade, vou cursar o que eu gosto',
       'sou muito grata a você',
       'corrupção é algo me decepciona, sempre que vejo notícias sobre isso fico mal',
       'a covid matou muitas pessoas no mundo todo, isso é algo muito tr

In [9]:
sentimentos = base_de_dados['Sentimento'].values
sentimentos

array(['tristeza', 'tristeza', 'tristeza', 'felicidade', 'felicidade',
       'felicidade', 'felicidade', 'raiva', 'raiva', 'felicidade',
       'felicidade', 'felicidade', 'tristeza', 'tristeza', 'tristeza',
       'tristeza', 'raiva', 'raiva', 'tristeza', 'tristeza', 'tristeza',
       'raiva', 'raiva', 'raiva', 'felicidade', 'felicidade', 'tristeza',
       'tristeza', 'tristeza', 'tristeza', 'felicidade', 'felicidade',
       'raiva', 'raiva', 'felicidade', 'felicidade', 'felicidade',
       'felicidade', 'felicidade', 'felicidade', 'felicidade',
       'felicidade', 'raiva', 'raiva', 'raiva', 'raiva', 'tristeza',
       'tristeza', 'tristeza', 'tristeza', 'raiva', 'raiva', 'raiva',
       'raiva', 'raiva', 'raiva', 'raiva', 'raiva', 'raiva', 'raiva',
       'tristeza', 'tristeza', 'tristeza', 'tristeza', 'tristeza',
       'tristeza'], dtype=object)


Para treinar o modelo será utilizada a abordagem Bag of Words e o algoritmo Naive Bayes Multinomial

Bag of Words, na prática, cria um vetor com cada uma das palavras da frase, depois, calcula a frequência em que essas palavras ocorrem, para então classificar/treinar o modelo

In [10]:
vectorizer = CountVectorizer(analyzer="word")
frequencia = vectorizer.fit_transform(frases)
# A função fit_transform faz o processo de ajustar o modelo, aprender o vocabulário, 
# e transformar os dados de treinamento em feature vectors (vetores com frequêcia das palavras)

modelo = MultinomialNB()
modelo.fit(frequencia, sentimentos)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### Testando o modelo

In [11]:
frases_teste = ["Não quero saber de nada, eu te odeio!",
                "Sinto sua falta",
                "Finalmente consegui o estágio que queria!",
                "Estou feliz por você estar bem",
                "Eu não suporto aquela pessoa.",
                "Meu primo pegou corona vírus, mas sobreviveu, ainda bem!",
                "Não estou me sentindo bem hoje, quero ficar sozinha"]

frequencia_testes = vectorizer.transform(frases_teste)
print(modelo.predict(frequencia_testes))

['raiva' 'tristeza' 'felicidade' 'felicidade' 'raiva' 'felicidade'
 'tristeza']


### Avaliando o modelo

- Fazendo a validação cruzada do modelo através da função `cross_val_predict`, ele é dividido em 19 partes, treinado em 18 e testado em 1



In [12]:
resultados = cross_val_predict(modelo, frequencia, sentimentos, cv = 19) #cv corresponde ao menor número de membros existentes na classe 'Sentimento', nesse caso, seria o valor total de 'felicidade'
resultados

array(['tristeza', 'tristeza', 'raiva', 'tristeza', 'felicidade',
       'felicidade', 'felicidade', 'raiva', 'raiva', 'tristeza', 'raiva',
       'felicidade', 'tristeza', 'tristeza', 'raiva', 'felicidade',
       'raiva', 'raiva', 'tristeza', 'tristeza', 'tristeza', 'raiva',
       'raiva', 'raiva', 'felicidade', 'tristeza', 'tristeza', 'raiva',
       'tristeza', 'felicidade', 'felicidade', 'felicidade', 'raiva',
       'raiva', 'felicidade', 'tristeza', 'felicidade', 'felicidade',
       'felicidade', 'felicidade', 'felicidade', 'felicidade', 'raiva',
       'tristeza', 'raiva', 'raiva', 'tristeza', 'tristeza', 'tristeza',
       'tristeza', 'raiva', 'raiva', 'raiva', 'raiva', 'raiva', 'raiva',
       'raiva', 'raiva', 'raiva', 'raiva', 'felicidade', 'tristeza',
       'tristeza', 'tristeza', 'tristeza', 'felicidade'], dtype='<U10')

- Medidas de validação do modelo utilizando a função `classification_report`

In [13]:
target = ['felicidade', 'raiva', 'tristeza']
print(classification_report(sentimentos, resultados, target))

              precision    recall  f1-score   support

  felicidade       0.78      0.74      0.76        19
       raiva       0.85      0.96      0.90        23
    tristeza       0.77      0.71      0.74        24

    accuracy                           0.80        66
   macro avg       0.80      0.80      0.80        66
weighted avg       0.80      0.80      0.80        66




precision = true positive / (true positive + false positive)

recall = true positive / (true positive + false negative)

f1-score = 2 ((precision recall) / (precision + recall))

- Gerando uma matriz de confusão para poder visualizar o desempenho do algoritmo

In [14]:
print(pd.crosstab(sentimentos, resultados, rownames = ["Real"], colnames=["Predito"], margins=True))

Predito     felicidade  raiva  tristeza  All
Real                                        
felicidade          14      1         4   19
raiva                0     22         1   23
tristeza             4      3        17   24
All                 18     26        22   66


- Calculando a acurácia (proximidade de um resultado com o seu valor de referência real) do modelo



In [15]:
accuracy_score(sentimentos, resultados)

0.803030303030303

Uma acurácia de nível entre 30 e 90% apresenta resultados de média precisão. Sendo assim, dependendo da informação a ser validada e do nível de risco que um erro ou engano representaria para seu negócio, soluções com esse nível de acurácia podem valer a pena.